# Fake News Classifier Using LSTM

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
# dropping nan values:
df=df.dropna()

In [7]:
df.shape

(18285, 5)

In [8]:
## Extracting independent and dependent data:
X = df.drop('label',axis=1)   ## Independent feature
Y = df['label']               ## Dependent features

In [9]:
print(f"X's shape: {X.shape}")
print(f"Y's shape: {Y.shape}")

X's shape: (18285, 4)
Y's shape: (18285,)


In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.14.0'

In [12]:
## required libraries:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
## Vocabulary size;
voc_size = 5000

# One_hot representation

In [14]:
messages = X.copy()

In [15]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Prashant
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [19]:
## Data preprocessing:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(words) for words in review if words not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [22]:
onehot_repr

[[3473, 4159, 1330, 759, 758, 3978, 479, 1779, 4713, 3904],
 [1065, 2586, 2142, 4888, 3614, 3043, 4711],
 [2209, 3572, 3600, 2688],
 [3527, 1732, 3514, 945, 3983, 904],
 [2044, 3614, 3412, 1927, 3800, 1915, 3614, 2343, 1843, 541],
 [977,
  1115,
  1563,
  996,
  714,
  3157,
  1713,
  4084,
  791,
  3206,
  330,
  280,
  2105,
  855,
  4711],
 [2974, 3623, 2278, 3174, 3142, 4948, 2707, 1101, 3411, 479, 2807],
 [1289, 2290, 4366, 1846, 2414, 3514, 3157, 4305, 3411, 479, 2807],
 [76, 1623, 1077, 871, 1643, 3793, 409, 1406, 3157, 1379],
 [4095, 4659, 14, 2806, 515, 3203, 3929, 2419],
 [4927, 368, 4258, 3280, 4193, 991, 919, 3726, 1056, 212, 2362],
 [945, 3023, 758, 3793, 3157, 2414],
 [296, 3525, 3124, 4435, 3588, 228, 3744, 2710, 2816],
 [3771, 2959, 3470, 4298, 739, 3942, 4778, 3411, 479, 2807],
 [975, 352, 4879, 735, 1877, 3411, 479, 2807],
 [3575, 2469, 2410, 4124, 4819, 4743, 4557, 2890, 1450, 2462],
 [1446, 2757, 2586],
 [4888, 265, 1995, 3345, 3157, 3033, 3527, 4711],
 [2824, 3285,

In [23]:
## Embedding representation:
## Making each sentence of equal size
sent_size = 20
embedded_doc = pad_sequences(onehot_repr, padding='pre', maxlen=sent_size)

In [24]:
embedded_doc

array([[   0,    0,    0, ..., 1779, 4713, 3904],
       [   0,    0,    0, ..., 3614, 3043, 4711],
       [   0,    0,    0, ..., 3572, 3600, 2688],
       ...,
       [   0,    0,    0, ..., 3411,  479, 2807],
       [   0,    0,    0, ...,  494, 3679, 4014],
       [   0,    0,    0, ..., 3346, 4693, 2209]])

# Model Creating:

In [25]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_size))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
## converting data into numpy arrays:
x_final = np.array(embedded_doc)
y_final = np.array(Y)

In [27]:
print(f"x_final's shape: {x_final.shape}")
print(f"y_final's shape: {y_final.shape}")

x_final's shape: (18285, 20)
y_final's shape: (18285,)


In [28]:
## Train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

# Model Training:

In [29]:
import time 

In [30]:
start_time = time.time()
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)
end_time = time.time()
print("elapsed time: ", end_time-start_time)

Epoch 1/10
192/192 [==============================] - 7s 24ms/step - loss: 0.3335 - accuracy: 0.8535 - val_loss: 0.1935 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1361 - accuracy: 0.9482 - val_loss: 0.1995 - val_accuracy: 0.9163
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0935 - accuracy: 0.9666 - val_loss: 0.2209 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0644 - accuracy: 0.9791 - val_loss: 0.2469 - val_accuracy: 0.9162
Epoch 5/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0407 - accuracy: 0.9868 - val_loss: 0.2945 - val_accuracy: 0.9117
Epoch 6/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0211 - accuracy: 0.9938 - val_loss: 0.3733 - val_accuracy: 0.9102
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0131 - accuracy: 0.9968 - val_loss: 0.5042 - val_accuracy: 0.9127

In [31]:
## predicting y_test
# y_pred = np.argmax(model.predict(x_test), axis=1)
y_predict = (model.predict(x_test)>0.5).astype('int32')

189/189 [==============================] - 2s 5ms/step


In [33]:
y_predict

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [36]:
acc_score = accuracy_score(y_predict,y_test)
conf_matrix = confusion_matrix(y_predict, y_test)
print("Accuracy Score: ", acc_score)
print("\n\nConfusion Matrix: \n", conf_matrix)

Accuracy Score:  0.9073736536868269


Confusion Matrix: 
 [[3085  225]
 [ 334 2391]]


## Adding Dropout layer

In [37]:
from tensorflow.keras.layers import Dropout

In [38]:
## Model 
mod = Sequential()
embb_vect_features = 40
mod.add(Embedding(voc_size, embb_vect_features, input_length=sent_size))
mod.add(Dropout(0.3))
mod.add(LSTM(100))
mod.add(Dropout(0.3))
mod.add(Dense(1, activation='sigmoid'))
mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
start_time = time.time()
mod.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)
end_time = time.time()
print("elapsed time: ", end_time-start_time)

Epoch 1/10
192/192 [==============================] - 7s 25ms/step - loss: 0.3404 - accuracy: 0.8363 - val_loss: 0.1950 - val_accuracy: 0.9137
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1440 - accuracy: 0.9436 - val_loss: 0.1905 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1042 - accuracy: 0.9629 - val_loss: 0.2080 - val_accuracy: 0.9178
Epoch 4/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0790 - accuracy: 0.9727 - val_loss: 0.2504 - val_accuracy: 0.9163
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0555 - accuracy: 0.9808 - val_loss: 0.3012 - val_accuracy: 0.9117
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0431 - accuracy: 0.9840 - val_loss: 0.2847 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0275 - accuracy: 0.9904 - val_loss: 0.3531 - val_accuracy: 0.9084

In [40]:
y_pre = (mod.predict(x_test)>0.5).astype('int32')

189/189 [==============================] - 1s 5ms/step


In [41]:
acc = accuracy_score(y_pre, y_test)
print(acc)

0.908036454018227


In [42]:
con_matr = confusion_matrix(y_pre, y_test)

In [43]:
con_matr

array([[3099,  235],
       [ 320, 2381]], dtype=int64)